In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output as co
import seaborn as sns
import os

In [2]:
from tools import preprocess

In [6]:
preprocess().clean_series

<bound method preprocess.clean_series of <tools.preprocess object at 0x000002119499A2B0>>

---
# Engenharia de Dados

### Novas variáveis:
1. Valores passados da mesma série - dois esquemas: "n antes" (cada coluna é a area n anos antes) ou cada coluna um ano fixo (nan para anos futuros).
2. Variações dos valores passados da mesma série.
3. Valores passados de séries diferentes.
4. Tabela com séries concatenadas com atraso de um ano para cada série - Exemplo: tabela com colunas "ano", "variavel alvo" (de uma série), demais séries como colunas, porém atrasadas em um ano
5. Valores passados de dados externos
6. Variações dos valores passados de dados externos

## 0. Carregando dados

In [4]:
def load_series(path='series/'):
    filenames = os.listdir(path)
    series, cnt, n_files = {}, 0, len(filenames)
    freq = range(1, n_files, 10)
    for filename in filenames:
        cnt+=1
        if cnt in freq: print(f'Files loaded: {cnt}/{n_files}'); co(wait=True)
        series[filename[:-4]] = pd.read_csv(path+filename, index_col=0)
    print(f'Done! Loaded {n_files} files.')
    return series

In [5]:
series = load_series()

Done! Loaded 1712 files.


In [4]:
data = pd.read_csv('data/historical-database-clean.csv')

## 1. Concatenando séries temporais como colunas

In [5]:
keys = series.keys()
series_df = pd.concat([series[key]['area'] for key in keys], 1)
series_df.columns = keys
series_df.head(3)

,0307883b5d063703-Açaí,0307883b5d063703-Beans,0307883b5d063703-Cassava,0307883b5d063703-Cocoa,0307883b5d063703-Corn,0307883b5d063703-Livestock,0307883b5d063703-Others-permanent,0307883b5d063703-Others-temporary,0307883b5d063703-Palm oil,0307883b5d063703-Rice,...,fda49275fbf5caf6-Cassava,fda49275fbf5caf6-Cocoa,fda49275fbf5caf6-Corn,fda49275fbf5caf6-Livestock,fda49275fbf5caf6-Others-permanent,fda49275fbf5caf6-Others-temporary,fda49275fbf5caf6-Palm oil,fda49275fbf5caf6-Rice,fda49275fbf5caf6-Sorghum,fda49275fbf5caf6-Soy
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Observação: Nº de séries com zero células vazias não corresponde com avaliação da explorção de dados

In [132]:
#series_df.to_csv('data/isolated-series.csv', index=True)
series_df = pd.read_csv('data/isolated-series.csv', index_col=0)

## 2. Valores passados da mesma série